In [1]:
import numpy as np
import cv2
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf 
import pyrealsense2 as rs
import serial
import time 

%matplotlib qt


# Load saved Keras model

In [2]:
saved_model_path = r'C:\Users\77bis\Desktop\CS598-FinalProject\Code\Leo\alex_saved_model'
new_model = tf.keras.models.load_model(saved_model_path)

new_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 60, 80, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 60, 80, 16)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 30, 40, 16)   0           activation[0][0]                 
______________________________________________________________________________________________

# Predict with full model

Full model takes approximately 24ms to run for each stream of force and image data, with error of 3.7 deg.

In [ ]:
i_start = 1
i_end = 20000


# Setup standardization
sc_X2 = StandardScaler()
sc_y = StandardScaler()


# Read force data 
force_file_path = r'C:\Users\77bis\Box\CS598 - Final Project\Preliminary Data V5\Test_Subject_leo\test35\fcss_processed_leo_test35_11_25_2020.txt'
f_data_raw = pd.read_csv(force_file_path)
f_data_all = sc_X2.fit_transform(f_data_raw.values)
f_data_i = f_data_all[i_start:i_end, :]


# Read ground truth data
qtm_file_path = r'C:\Users\77bis\Box\CS598 - Final Project\Preliminary Data V5\Test_Subject_leo\test35\qtm_processed_leo_test35_11_25_2020.txt'
qtm_data_raw = pd.read_csv(qtm_file_path)
qtm_data_all = sc_y.fit_transform(qtm_data_raw.values[:,0].reshape(-1,1))
qtm_data_i = qtm_data_all[i_start:i_end, :]

img_file_path = r'C:\Users\77bis\Box\CS598 - Final Project\Preliminary Data V5\Test_Subject_leo\test35\depth_processed_leo_test35.avi'
cap = cv2.VideoCapture(img_file_path)
image_data_i = []

for data_idx in range(i_start, i_end):
    # Read depth image data
    cap.set(1, data_idx-1)
    res, frame = cap.read()
    gray_frame = frame/255.0
    ex_img_data = cv2.resize(gray_frame, \
                                        (80, 60),\
                                        interpolation = cv2.INTER_NEAREST)
    image_data_i.append(ex_img_data) 


y_pred = new_model.predict([np.array(image_data_i), f_data_i])

In [ ]:
%timeit y_pred = new_model.predict([np.array(image_data_i), f_data_i])

In [ ]:
y_pred_2 = sc_y.inverse_transform(y_pred)
y_truth_2 = sc_y.inverse_transform(qtm_data_i)
N = len(qtm_data_i)
plot_i = np.arange(0, N)

truth = plt.plot(plot_i, y_truth_2, label = 'truth')
pred = plt.plot(plot_i, y_pred_2, label = 'pred')
plt.legend()
plt.show()

avg_e = (abs(y_truth_2 - y_pred_2))
print(avg_e.mean())

# Convert to Lite Model

In [3]:
# Convert full model to lite model 
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)

lite_model_content = converter.convert()


# Save lite model
lite_model_path = r'C:\Users\77bis\Desktop\CS598-FinalProject\Code\Leo\test_lite_model'
with open(lite_model_path, "wb") as f:
    f.write(lite_model_content)

In [4]:
# Interpret lite model 
interpreter = tf.lite.Interpreter(model_content=lite_model_content)


# This little helper wraps the TF Lite interpreter as a numpy-to-numpy function.
def lite_model(image, force):
    interpreter.allocate_tensors()
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], image)
    interpreter.set_tensor(interpreter.get_input_details()[1]['index'], force)
    interpreter.invoke()
    return interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

# Load Tensorflow Lite model


In [4]:
# Save lite model
lite_model_path = r'C:\Users\77bis\Desktop\CS598-FinalProject\Code\Leo\test_lite_model'


# Interpret lite model 
interpreter = tf.lite.Interpreter(model_path = lite_model_path)


# This little helper wraps the TF Lite interpreter as a numpy-to-numpy function.
def lite_model(image, force):
    interpreter.allocate_tensors()
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], image)
    interpreter.set_tensor(interpreter.get_input_details()[1]['index'], force)
    interpreter.invoke()
    return interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

# Predict Lite Model using existing Dataset
Lite model takes about 1ms to run each stream of force and image data with error of 3.5 deg

In [13]:
def compute_angle_xy(f_data):
    d1 = 0.19
    d2 = 0.08
    d3 = 0.19
    
    offset_x = -0.8768
    slope_x = -0.4964

    offset_y = 31.2436
    slope_y = -0.4404
    
    g = 9.81
    
    Fz1 = f_data[0] 
    Fz2 = f_data[1]
    Fz3 = f_data[2]
    Fx1 = f_data[3]
    Fx2 = f_data[4]
    Fy = f_data[5]
    
    Mx = ((Fz2 - Fz3) * d3 + Fy * d2) * g
    My = ((Fz2 + Fz3 - Fz1) * d1 + (Fx1 + Fx2) * d2) * g

    theta_x = offset_x + slope_x * Mx
    theta_y = offset_y + slope_y * My


    return theta_x, theta_y

In [40]:
compute_angle_xy(ex_f_data_raw)

(11.904659594800004, -0.23687285840000527)

In [44]:
f_data_i_raw.shape

(3000, 6)

In [53]:
# Setup standardization
sc_X2 = StandardScaler()
sc_y = StandardScaler()



# Read force data 
force_file_path = r'C:\Users\77bis\Box\CS598 - Final Project\Preliminary Data V5\Test_Subject_leo\test35\fcss_processed_leo_test35_11_25_2020.txt'
f_data_raw = pd.read_csv(force_file_path)
f_data_all = sc_X2.fit_transform(f_data_raw.values)
f_data_all_raw = f_data_raw.values


N = len(f_data_all)
i_start = 1
i_end = N

f_data_i = f_data_all[i_start:i_end, :]


# Read ground truth data
qtm_file_path = r'C:\Users\77bis\Box\CS598 - Final Project\Preliminary Data V5\Test_Subject_leo\test35\qtm_processed_leo_test35_11_25_2020.txt'
qtm_data_raw = pd.read_csv(qtm_file_path)
qtm_data_all = sc_y.fit_transform(qtm_data_raw.values[:,0].reshape(-1,1))
qtm_data_i = qtm_data_raw.values[i_start:i_end, :]

img_file_path = r'C:\Users\77bis\Box\CS598 - Final Project\Preliminary Data V5\Test_Subject_leo\test35\depth_processed_leo_test35.avi'
cap = cv2.VideoCapture(img_file_path)

# prediction for ML
y_pred_x = np.zeros((i_end - i_start,1))
y_pred_y = np.zeros((i_end - i_start,1))
y_pred_z = np.zeros((i_end - i_start,1))


# Save prediction
f = open(r"C:\Users\77bis\Box\CS598 - Final Project\Preliminary Data V5\Test_Subject_leo\test35\ml_pred_leo_test35.txt", "x")

for data_idx in range(i_start, i_end):
    ex_f_data_raw = f_data_all_raw[data_idx-2,:]
    ex_f_data = f_data_all[data_idx-2,:]


    # Read depth image data
    cap.set(1, data_idx-1)
    res, frame = cap.read()
    gray_frame = frame/255.0
    ex_img_data = cv2.resize(gray_frame, \
                                        (80, 60),\
                                        interpolation = cv2.INTER_NEAREST)
    # cv2.imshow('s',ex_img_data)
    # cv2.waitKey(0)  
    # cv2.destroyAllWindows()  

    ex_qtm_data = qtm_data_all[data_idx - 2]
    
    y_pred_temp = lite_model([np.array(ex_img_data, dtype='float32')], [np.array(ex_f_data, dtype='float32')])[0]
    
    y_pred_z[data_idx-i_start] = y_pred_temp[0]
    
    theta_x, theta_y = compute_angle_xy(ex_f_data_raw)
    
    y_pred_x[data_idx - i_start]  = theta_x
    y_pred_y[data_idx - i_start]  = theta_y
    
    f.write(str(theta_x) + ',' + str(theta_y) + ',' + str(sc_y.inverse_transform(y_pred_temp)[0]) + '\n')

    
f.close()

In [54]:
y_pred_z_2 = sc_y.inverse_transform(y_pred_z)
y_truth_z_2 = qtm_data_i[:,0]
N = len(qtm_data_i)
plot_i = np.arange(0, N)

plt.figure(1)
truth = plt.plot(plot_i, y_truth_z_2, label = 'truth')
pred = plt.plot(plot_i, y_pred_z_2, label = 'pred')
plt.legend()


y_truth_x_2 = qtm_data_i[:,2]

plt.figure(2)
truth = plt.plot(plot_i, y_truth_x_2, label = 'truth')
pred = plt.plot(plot_i, y_pred_x, label = 'pred')
plt.legend()


y_truth_y_2 = qtm_data_i[:,1]

plt.figure(3)
truth = plt.plot(plot_i, y_truth_y_2, label = 'truth')
pred = plt.plot(plot_i, y_pred_y, label = 'pred')
plt.legend()

plt.show()

avg_e = (abs(y_truth_z_2 - y_pred_z_2))
print(avg_e.mean())

13.798475485935807


# Predict using Live Data

In [ ]:

# Setup:
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)


# Create an object to read  
# from camera 
video = cv2.VideoCapture(0) 
   
# We need to check if camera 
# is opened previously or not 
if (video.isOpened() == False):  
    print("Error reading video file") 
    
# We need to set resolutions. 
# so, convert them from float to integer. 
frame_width = int(video.get(3)) 
frame_height = int(video.get(4)) 
   
size = (frame_width, frame_height) 



# Below VideoWriter object will create 
# a frame of above defined The output  
# is stored in 'filename.avi' file. 
colorizer = rs.colorizer()
colorizer.set_option(rs.option.color_scheme, 3)
mag_deci = 4
decimation = rs.decimation_filter(mag_deci)
threshold = rs.threshold_filter(0.15, 1.6)

# n_trial = 30
# depthwriter = cv2.VideoWriter('fcss_leo_depth_test'+str(n_trial)+'.avi',  
#                          cv2.VideoWriter_fourcc(*'DIVX'), 
#                          30, (int(frame_width/mag_deci), int(frame_height/mag_deci)), 0) 


print("type anything to start")
start_test = input()


addr = "COM4" ## serial port to read data from
baud = 115200 ## baud rate for instrument

ser = serial.Serial(
    port = addr,\
    baudrate = baud)

# fcss_filename = "fcss_leo_load_test"+str(n_trial)+".csv"
# datafile = open(fcss_filename, 'a')


sc_X2 = StandardScaler()
sc_y = StandardScaler()
qtm_file_path = r'C:\Users\77bis\Box\CS598 - Final Project\Preliminary Data V5\Test_Subject_leo\test35\qtm_processed_leo_test35_11_25_2020.txt'
qtm_data_raw = pd.read_csv(qtm_file_path)
qtm_data_all = sc_y.fit_transform(qtm_data_raw.values[:,0].reshape(-1,1))


# Start streaming
pipe.start(cfg)
start_time = cv2.getTickCount()
prev_time = 0
count = 0
count_total = 0 


try:
    while True:
#         print((cv2.getTickCount() - start_time)/cv2.getTickFrequency())
#         try:
            prev_time = cv2.getTickCount()
            ser_bytes = ser.readline()
            decoded_bytes = (ser_bytes[0:len(ser_bytes)-2].decode("utf-8").split(","))

            write_line = ",".join(decoded_bytes) + str("\n")
            values = [float(i) for i in write_line.split(',')]
            force_data = sc_X2.fit_transform(np.array(values[1:], dtype = 'float32').reshape(-1,1))
#             datafile.write(write_line)
            
            
            # Wait for a coherent pair of frames: depth and color
            frames = pipe.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            
            if not depth_frame:
                continue
            
            # Decimation 
            depth_frame = decimation.process(depth_frame)

            # Thresholding
            depth_frame = threshold.process(depth_frame)
        
            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            depth_colormap = np.asanyarray(colorizer.colorize(depth_frame).get_data())     
            key = cv2.waitKey(1)
            
            # Resize the CV frame to fit the decimated frame
            depth_data = cv2.resize(depth_colormap, (80, 60))/255
            
         
            # Save depth camera frame 
#             depthwriter.write(depth_save)
            
            # Show depth camera 
#             cv2.imshow('Depth', depth_data)
            
            y_pred_temp = lite_model([np.array(depth_data, dtype='float32')], [force_data[:,0]])[0]
            print(sc_y.inverse_transform(y_pred_temp))
            
            # Press esc or 'q' to close the image window
            if key & 0xFF == ord('q') or key == 27:
                cv2.destroyAllWindows()
                video.release() 
#                 depthwriter.release()
                break 
#             count_total = count_total+ 1  
#             el_t = (cv2.getTickCount() - prev_time)/cv2.getTickFrequency()
#             if el_t > 0.04:
#                 count = count + 1
#                 print(el_t)
#             prev_time = cv2.getTickCount()
            
                
#         except:
#             pass
#             break

finally:
    ser.close()
#     datafile.close()
    pipe.stop()
    cv2.destroyAllWindows()
    video.release() 
#     depthwriter.release()
#     print(count/count_total * 100)

In [ ]:
y_pred_temp = lite_model([np.array(depth_data, dtype='float32')], [force_data[:,0]])[0]

In [ ]:
y_pred_temp

In [ ]:
f = open("demofile2.txt", "a")
f.write("Now the file has more content!")
f.close()
